In [1]:
import torch
import torch.nn as nn

/home/kbh/.conda/envs/dnn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
class ResBlock(nn.Module):
    def __init__(self, n_channel,norm="BatchNorm2d",dropout=0.0,activation="PReLU"):
        super().__init__()

        ## Normalization
        if norm == "BatchNorm2d": 
            bn= nn.BatchNorm2d
        else :
            raise Exception("ERROR::ResBlock: unknown nromalization - {}".format(norm))

        if activation == 'PReLU':
            acti = nn.PReLU()
        else :
            raise Exception("ERROR::ResBlokc:Unknown activation type " + str(activation))

        dilations= [
            (1,1),
            (1,2),
            (1,4),
            (1,8),
            (1,16)
        ]

        self.layers=[]
        for i in range(5):
            self.layers.append(nn.Sequential(
                    nn.Conv2d(n_channel,n_channel,
                    (3,3),(1,1), # <- kernel should be (3,2)
                    dilations[i],dilations[i],n_channel),
                    nn.Conv2d(n_channel,n_channel,
                    (1,1)),
                    acti,
                    bn(n_channel)
                    )
                )

    def forward(self,x):

        residual = self.layers[0](x)
        print(residual.shape)
        for i in range(1,5):
            y = self.layers[i](residual)
            print(y.shape)
            residual += y
            

        return residual

In [52]:
m = ResBlock(30)
x = torch.rand(2,30,257,125)
y = m(x)
print(y.shape)

torch.Size([2, 30, 257, 125])
torch.Size([2, 30, 257, 125])
torch.Size([2, 30, 257, 125])
torch.Size([2, 30, 257, 125])
torch.Size([2, 30, 257, 125])
torch.Size([2, 30, 257, 125])
